In [1]:
import os, sys, subprocess, json


os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]


os.environ["PYSPARK_PYTHON"] = sys.executable




print("JAVA_HOME =", os.environ.get("JAVA_HOME"))
print("java -version ->")
print(subprocess.check_output(["bash","-lc","java -version"], stderr=subprocess.STDOUT).decode())

JAVA_HOME = /opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home
java -version ->
openjdk version "17.0.16" 2025-07-15
OpenJDK Runtime Environment Homebrew (build 17.0.16+0)
OpenJDK 64-Bit Server VM Homebrew (build 17.0.16+0, mixed mode, sharing)



In [2]:
import pyspark
from pyspark.sql.functions import col, month, date_add, to_date,from_json
print(pyspark.__version__)

4.0.0


In [3]:
!java -version

openjdk version "17.0.16" 2025-07-15
OpenJDK Runtime Environment Homebrew (build 17.0.16+0)
OpenJDK 64-Bit Server VM Homebrew (build 17.0.16+0, mixed mode, sharing)


In [4]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
      .appName("")
      .config("spark.sql.shuffle.partitions", "10")
      .config("spark.ui.showConsoleProgress", "false")
      .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0")
      .getOrCreate())


:: loading settings :: url = jar:file:/Users/rohan/Desktop/CS/projects/kafka_project/venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/rohan/.ivy2.5.2/cache
The jars for the packages stored in: /Users/rohan/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8446878c-0ec4-4fcd-90f5-895681d2895a;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.0 in central
	found org.apache.kafka#kafka-clients;3.9.0 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found com.google.code.findbugs

In [5]:
df = (spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "stations-json")
    .option("startingOffsets", "earliest")
    .load())

In [6]:
df.isStreaming

True

In [7]:
schema = "station STRING, date DATE, degrees DOUBLE, raining INT"

records = (
    df.select(col("key").cast("string").alias("key"),
              col("value").cast("string").alias("json"))
      .select("key", from_json(col("json"), schema).alias("value"))
      .select("key", "value.*")   # expands to date, station, degrees, raining
)

In [8]:
from pyspark.sql.functions import min as fmin, max as fmax, avg as favg, count as fcount
stats = (records.groupBy("station")
           .agg(
               fmin("date").alias("start"),
               fmax("date").alias("end"),
               fcount("*").alias("measurements"),
               favg("degrees").alias("avg"),
               fmax("degrees").alias("max")
           )
           .select("station", "start", "end", "measurements", "avg", "max")  # order columns
           .orderBy("station"))

q_stats = (stats.writeStream
         .format("console")
         .outputMode("complete")            # print full table every trigger
         .trigger(processingTime="5 seconds")
         .option("truncate", "false")
         .option("numRows", 1000)
         .start())

25/08/29 11:52:45 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/5r/89jndgws4q39x08qhlxlm_gr0000gn/T/temporary-46210cf7-6106-4804-8a55-1181b508bb6b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/08/29 11:52:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [9]:
from pyspark.sql.functions import col, month, date_add, to_timestamp


today = records.select("station","date","raining") \
               .withColumn("event_time", to_timestamp(col("date"))) \
               .withWatermark("event_time", "3 days")

yesterday = records.select(
                col("station"),
                date_add(col("date"), 1).alias("date"),
                col("degrees").alias("sub1degrees"),
                col("raining").alias("sub1raining")
           ).withColumn("event_time", to_timestamp(col("date"))) \
            .withWatermark("event_time", "3 days")

two_days = records.select(
                col("station"),
                date_add(col("date"), 2).alias("date"),
                col("degrees").alias("sub2degrees"),
                col("raining").alias("sub2raining")
           ).withColumn("event_time", to_timestamp(col("date"))) \
            .withWatermark("event_time", "3 days")


training_rows = (
    today
      .join(
          yesterday.select("station","event_time","sub1degrees","sub1raining"),
          ["station","event_time"], "left"
      )
      .join(
          two_days.select("station","event_time","sub2degrees","sub2raining"),
          ["station","event_time"], "left"
      )
      .withColumn("month", month(col("date")))
      .select("station","date","raining","month",
              "sub1degrees","sub1raining","sub2degrees","sub2raining")
)


out_path = "rain_training_parquet"
ckpt     = "chk_rain_training"

sink = (
    training_rows.repartition(1)
                 .writeStream
                 .format("parquet")
                 .option("checkpointLocation", ckpt)  
                 .outputMode("append")
                 .trigger(processingTime="1 minute")
                 .start(out_path)                    
)

25/08/29 11:52:47 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/08/29 11:52:48 ERROR FileFormatWriter: Aborting job c70260b4-527f-401a-814a-fb6ff967a02b.
org.apache.spark.sql.kafka010.KafkaIllegalStateException: Some data may have been lost because they are not available in Kafka any more;
either the data was aged out by Kafka or the topic may have been deleted before all the data in the
topic was processed.
If you don't want your streaming query to fail on such cases, set the source option failOnDataLoss to false.
Reason: Partition stations-json-2 offset was changed from 453 to 33.
	at org.apache.spark.sql.kafka010.KafkaExceptions$.partitionOffsetChanged(KafkaExceptions.scala:151)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderAdmin.$anonfun$getOffsetRangesFromResolvedOffsets$10(KafkaOffsetReaderAdmin.scala:509)
	at org.apache.spark.sql.kafka010.KafkaMicroBatchStream.reportDataLoss(KafkaMicroBatchSt

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+----------+----------+------------+------------------+------------------+
|station|start     |end       |measurements|avg               |max               |
+-------+----------+----------+------------+------------------+------------------+
|A      |2000-01-01|2000-01-03|3           |31.714536648388023|35.99886859506166 |
|B      |2000-01-01|2000-01-02|2           |19.41692640397019 |22.38389483821583 |
|C      |2000-01-01|2000-01-02|2           |45.51282292449058 |46.6782408306873  |
|D      |2000-01-01|2000-01-02|2           |13.693325465649643|16.533667605999277|
|E      |2000-01-01|2000-01-02|2           |28.01508294119001 |28.619490102322693|
|F      |2000-01-01|2000-01-02|2           |31.289931268409333|35.829869020157574|
|G      |2000-01-01|2000-01-02|2           |38.63036756812535 |42.352589804873205|
|H      |2000-01-01|2000-01-02|2           |34.27227636592002 |34.4248497

In [10]:
# for q in spark.streams.active:
#     q.stop()

-------------------------------------------
Batch: 2
-------------------------------------------
+-------+----------+----------+------------+------------------+------------------+
|station|start     |end       |measurements|avg               |max               |
+-------+----------+----------+------------+------------------+------------------+
|A      |2000-01-01|2000-01-03|3           |31.714536648388023|35.99886859506166 |
|B      |2000-01-01|2000-01-03|3           |18.865810571193062|22.38389483821583 |
|C      |2000-01-01|2000-01-03|3           |47.77427201865643 |52.29717020698815 |
|D      |2000-01-01|2000-01-03|3           |13.643608611122508|16.533667605999277|
|E      |2000-01-01|2000-01-03|3           |29.34043794162423 |31.99114794249268 |
|F      |2000-01-01|2000-01-03|3           |27.67662119949613 |35.829869020157574|
|G      |2000-01-01|2000-01-03|3           |38.33537804139353 |42.352589804873205|
|H      |2000-01-01|2000-01-03|3           |36.045604187859766|39.5922598

In [11]:

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.regression import Regressor, RegressionModel

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+----------+----------+------------+------------------+------------------+
|station|start     |end       |measurements|avg               |max               |
+-------+----------+----------+------------+------------------+------------------+
|A      |2000-01-01|2000-01-04|4           |31.264650048223924|35.99886859506166 |
|B      |2000-01-01|2000-01-04|4           |18.476028885189216|22.38389483821583 |
|C      |2000-01-01|2000-01-04|4           |49.398848303377484|54.27257715754064 |
|D      |2000-01-01|2000-01-04|4           |15.280967492443544|20.193044136406655|
|E      |2000-01-01|2000-01-04|4           |30.555224419802784|34.19958385433844 |
|F      |2000-01-01|2000-01-04|4           |27.814902242197867|35.829869020157574|
|G      |2000-01-01|2000-01-04|4           |38.29102094216404 |42.352589804873205|
|H      |2000-01-01|2000-01-04|4           |34.48606310935395 |39.5922598

In [12]:
data = spark.read.parquet("rain_training_parquet")
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
assembler = VectorAssembler(
    inputCols=["month","sub1degrees","sub1raining","sub2degrees","sub2raining"],
    outputCol="features"
)
train_data = assembler.transform(train_data)
test_data  = assembler.transform(test_data)

-------------------------------------------
Batch: 6
-------------------------------------------
+-------+----------+----------+------------+------------------+------------------+
|station|start     |end       |measurements|avg               |max               |
+-------+----------+----------+------------+------------------+------------------+
|A      |2000-01-01|2000-01-04|4           |31.264650048223924|35.99886859506166 |
|B      |2000-01-01|2000-01-04|4           |18.476028885189216|22.38389483821583 |
|C      |2000-01-01|2000-01-04|4           |49.398848303377484|54.27257715754064 |
|D      |2000-01-01|2000-01-04|4           |15.280967492443544|20.193044136406655|
|E      |2000-01-01|2000-01-04|4           |30.555224419802784|34.19958385433844 |
|F      |2000-01-01|2000-01-04|4           |27.814902242197867|35.829869020157574|
|G      |2000-01-01|2000-01-04|4           |38.29102094216404 |42.352589804873205|
|H      |2000-01-01|2000-01-04|4           |34.48606310935395 |39.5922598

In [13]:
data = spark.read.parquet(out_path)

# Drop early rows that don’t yet have history
data = data.na.drop(subset=["sub1degrees", "sub1raining", "sub2degrees", "sub2raining"])

# Assemble features
assembler = VectorAssembler(
    inputCols=["month", "sub1degrees", "sub1raining", "sub2degrees", "sub2raining"],
    outputCol="features"
)
assembled = assembler.transform(data).select("station", "date", "raining", "features")

# Split & train
train_data, test_data = assembled.randomSplit([0.8, 0.2], seed=42)

dt = DecisionTreeClassifier(labelCol="raining", featuresCol="features", maxDepth=5)
dt_model = dt.fit(train_data)
print(dt_model.toDebugString)

# Evaluate
pred = dt_model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(
    labelCol="raining", predictionCol="prediction", metricName="accuracy"
)
accuracy = evaluator.evaluate(pred)

# Show the 2-number table like the spec
pred.selectExpr(
    "double(prediction = raining) as correct",
    "double(raining) as raining"
).groupBy().avg("correct","raining").show(truncate=False)

print("Accuracy:", accuracy)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_e91afb6c63bc, depth=5, numNodes=45, numClasses=2, numFeatures=5
  If (feature 2 <= 0.5)
   If (feature 0 <= 3.5)
    If (feature 3 <= 33.12982299395898)
     If (feature 1 <= 40.304383232893066)
      Predict: 0.0
     Else (feature 1 > 40.304383232893066)
      If (feature 0 <= 1.5)
       Predict: 1.0
      Else (feature 0 > 1.5)
       Predict: 0.0
    Else (feature 3 > 33.12982299395898)
     If (feature 0 <= 2.5)
      Predict: 0.0
     Else (feature 0 > 2.5)
      If (feature 1 <= 67.45038701134592)
       Predict: 0.0
      Else (feature 1 > 67.45038701134592)
       Predict: 1.0
   Else (feature 0 > 3.5)
    If (feature 1 <= 31.285077480407814)
     If (feature 4 <= 0.5)
      Predict: 0.0
     Else (feature 4 > 0.5)
      If (feature 0 <= 4.5)
       Predict: 0.0
      Else (feature 0 > 4.5)
       Predict: 1.0
    Else (feature 1 > 31.285077480407814)
     If (feature 0 <= 4.5)
      Predict: 1.0
     Else (feature 0

In [14]:
features_stream = training_rows.select(
    "station","date","month","sub1degrees","sub1raining","sub2degrees","sub2raining"
)

# Drop rows that don’t have full history yet
features_stream_clean = features_stream.na.drop(
    subset=["sub1degrees","sub1raining","sub2degrees","sub2raining"]
)

# Assemble with skip just in case
assembler_pred = VectorAssembler(
    inputCols=["month","sub1degrees","sub1raining","sub2degrees","sub2raining"],
    outputCol="features"
).setHandleInvalid("skip")

features_for_pred = assembler_pred.transform(features_stream_clean).select("station","date","features")

pred_stream = dt_model.transform(features_for_pred).select("station","date","prediction")

# (optional) filter to one station
# pred_stream = pred_stream.filter("station = 'A'")

q_pred = (pred_stream.writeStream
          .format("console")
          .outputMode("append")
          .option("truncate","false")
          .trigger(processingTime="5 seconds")
          .start())

25/08/29 11:53:21 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/5r/89jndgws4q39x08qhlxlm_gr0000gn/T/temporary-8e217583-988a-4c7f-9987-3121b0c38bd1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/08/29 11:53:21 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------+----------+----------+
|station|date      |prediction|
+-------+----------+----------+
|K      |2000-01-04|0.0       |
|C      |2000-01-03|0.0       |
|H      |2000-01-03|0.0       |
|H      |2000-01-04|0.0       |
|M      |2000-01-04|0.0       |
|N      |2000-01-03|0.0       |
|N      |2000-01-04|0.0       |
|B      |2000-01-03|0.0       |
|L      |2000-01-03|0.0       |
|E      |2000-01-04|0.0       |
|B      |2000-01-04|0.0       |
|C      |2000-01-04|0.0       |
|D      |2000-01-03|0.0       |
|G      |2000-01-04|0.0       |
|D      |2000-01-05|0.0       |
|I      |2000-01-03|0.0       |
|J      |2000-01-03|0.0       |
|C      |2000-01-05|0.0       |
|E      |2000-01-03|0.0       |
|G      |2000-01-03|0.0       |
+-------+----------+----------+
only showing top 20 rows
-------------------------------------------
Batch: 8
-------------------------------------------
+-------+----

In [15]:
for q in spark.streams.active: q.stop()

25/08/27 12:16:39 WARN DAGScheduler: Failed to cancel job group 4c31426c-c1d5-487e-9d69-484b79bfe2c3. Cannot find active jobs for it.
25/08/27 12:16:39 WARN DAGScheduler: Failed to cancel job group 4c31426c-c1d5-487e-9d69-484b79bfe2c3. Cannot find active jobs for it.


In [18]:
q_pred.isActive


False

-------------------------------------------
Batch: 32
-------------------------------------------
+-------+----------+----------+------------+------------------+-----------------+
|station|start     |end       |measurements|avg               |max              |
+-------+----------+----------+------------+------------------+-----------------+
|A      |2000-01-01|2000-03-08|68          |31.652024715046924|53.43912012050109|
|B      |2000-01-01|2000-03-08|68          |22.870967653965998|44.40074419586528|
|C      |2000-01-01|2000-03-08|68          |16.55702984241322 |42.56271453750142|
|D      |2000-01-01|2000-03-08|68          |22.607706280882088|38.24475903129374|
|E      |2000-01-01|2000-03-08|68          |49.13944375751521 |67.26244762498868|
|F      |2000-01-01|2000-03-07|67          |22.825837370913117|41.65781054832314|
|G      |2000-01-01|2000-03-07|67          |36.305611378090894|65.74825341925056|
|H      |2000-01-01|2000-03-07|67          |30.19538446833792 |50.92274448232121|
